In [ ]:
import pandas as pd
import numpy as np
import os

import pandas as pd
import numpy as np
from collections import Counter
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import math
from sklearn.preprocessing import MinMaxScaler
import scipy
from sklearn import metrics

import numpy as np
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr

import seaborn as sns
import matplotlib.pyplot as plt
import shap

from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from scipy.stats import linregress
from scipy import stats


In [ ]:
def plot_coef(coef_df_all, results_labels_df_all, title = ''):

    df = coef_df_all.iloc[1::, 0:2].sort_values(by='Estimate mean', ascending=False)
    
    colors_palette = get_bar_colors(df.reset_index().rename(columns={'index':'Features'}), X_cat_)
    sns.barplot(x='Estimate mean', y=df.index, data=df, palette=colors_palette, orient='h')
    plt.xlabel('Estimate mean')
    plt.ylabel('Variable')
    plt.xlim([0, 0.55])
    
    
    r2 = np.round(coef_df_all.loc['_intercept', 'R2'],2)
    f2 = np.round(coef_df_all.loc['_intercept', 'F2'],2)
    rmse = np.round(coef_df_all.loc['_intercept', 'rmse'],2)
    mde = np.round(coef_df_all.loc['_intercept', 'MDE'],2)
    mae = np.round(coef_df_all.loc['_intercept', 'MAE'],2)
    
    y_true = results_labels_df_all['y_labels']
    y_pred = results_labels_df_all['y_pred']
    
    correlacion, p_value = pearsonr(y_true, y_pred)
    
    
    plt.text( 0.23, df.shape[0]-6, r'$R^{2}$: ' + str(np.round(r2, 2)), va='center', ha='left')
    plt.text( 0.23, df.shape[0]-5, r'$F^{2}$: ' + str(np.round(f2, 2)), va='center', ha='left')
    plt.text( 0.23, df.shape[0]-4, f'corr: {correlacion:.2f}', va='center', ha='left')
    plt.text( 0.23, df.shape[0]-3, f'RMSE: {rmse:.2f}', va='center', ha='left')
    plt.text( 0.23, df.shape[0]-2, f'MDE: {mde:.2f}', va='center', ha='left')
    plt.text( 0.23, df.shape[0]-1, f'MAE: {mae:.2f}', va='center', ha='left')

    plt.title(title)


def plot_coef_v01(coef_df_for_plot, results_labels_df_for_plot, df_directions_for_plot):

    df_directions_for_plot = df_directions_for_plot.set_index('index')
    list_coef = list(coef_df_for_plot.index)
    list_coef.remove('_intercept')
    coef_df_for_orig = coef_df_for_plot.copy()
    coef_df_for_plot['coef_sign'] = 0
    
    for i in list_coef:
        sign =  df_directions_for_plot.loc[i, 'coef']
        if(sign<0):
            sign = -1
        else:
            sign = 1
    
        coef_df_for_plot.loc[i, 'coef_sign'] = coef_df_for_plot.loc[i, 'Estimate mean']*sign
    
    coef_df_for_plot['color'] = coef_df_for_plot['coef_sign'].apply(lambda x: 'red' if x > 0 else 'blue')
    
    coef_df_for_plot_sort = coef_df_for_plot.iloc[1::, :].sort_values(by='Estimate mean', ascending=False)
    coef_df_for_plot_sort = coef_df_for_plot_sort[['Estimate mean', 'Estimate std', 'coef_sign', 'color']]
    coef_df_for_plot_sort.reset_index(inplace = True)
    

    if(coef_df_for_plot_sort.loc[0, 'color'] == 'blue'):
        sns.barplot(x='coef_sign', y='index', data=coef_df_for_plot_sort, hue='color', dodge=False, palette=['royalblue', 'firebrick'], orient='h' ,legend=False)
    else:
        sns.barplot(x='coef_sign', y='index', data=coef_df_for_plot_sort, hue='color', dodge=False, palette=['firebrick', 'royalblue'], orient='h' ,legend=False)
        
    plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
    plt.xlim([-0.3, 0.3])
    plt.xlabel('Features importance')
    plt.ylabel('Features')
    
    r2 = np.round(coef_df_for_orig.loc['_intercept', 'R2'],4)
    f2 = np.round(coef_df_for_orig.loc['_intercept', 'F2'],4)
    rmse = np.round(coef_df_for_orig.loc['_intercept', 'rmse'],4)
    try:
        mde = np.round(coef_df_for_orig.loc['_intercept', 'MDE'],4)
    except:
        mde = 0
    mae = np.round(coef_df_for_orig.loc['_intercept', 'MAE'],4)
    
    y_true = results_labels_df_for_plot['y_labels']
    y_pred = results_labels_df_for_plot['y_pred']
    
    correlacion, p_value = pearsonr(y_true, y_pred)
    
    plt.title(r'$R^{2}$: ' + str(np.round(r2, 4)) + r'   $F^{2}$: ' + str(np.round(f2, 4)) + f'   $r$: {correlacion:.4f}' + f'\nRMSE: {rmse:.4f}' + f'   MDE: {mde:.4f}'+ f'   MAE: {mae:.4f}');
    
    plt.text( -0.20, 10, 'Protected\nFactors', va='center', ha='center', color='blue')
    plt.text( 0.20, 10, 'Risk\nFactors', va='center', ha='center', color='red')

    return coef_df_for_plot_sort

In [ ]:
def mean_directional_accuracy(y_true, y_pred):
    
    differences = np.array(y_pred) - np.array(y_true) 
    signs = np.sign(differences)
    mde = np.mean(signs)
    return mde

def mean_absolute_error(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    absolute_errors = np.abs(y_pred - y_true)
    mae = np.mean(absolute_errors)
    
    return mae

def coef_pval(coef_array_mean_, X_, y_, y_p):

    n = X_.shape[0]
    t = coef_tval(coef_array_mean_, X_, y_, y_p)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p


def coef_tval(coef_array_mean_, X_, y_, y_p):
    
    '''
        coef_tval for OLS of statsmodels
    '''
    
    a = np.array(coef_array_mean_[0][0]/ coef_se(X_, y_, y_p)[0])
    b = np.array(coef_array_mean_[1::].flatten() / coef_se(X_, y_, y_p)[1:])
    return np.append(a, b)


def coef_se(X_, y_, y_p):
    
    '''
        coef_se for OLS of statsmodels
    '''
    n = X_.shape[0]
    
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X_)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y_, y_p) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)

def directional_accuracy(predicted_values, true_values):

    predicted_values = np.array(predicted_values)
    true_values = np.array(true_values)    

    difference_direction = np.sign(predicted_values - true_values)    
    correct_direction_count = np.sum(difference_direction == 1) + np.sum(difference_direction == -1)
    
    directional_accuracy_score = correct_direction_count / len(predicted_values)
    
    return directional_accuracy_score



from sklearn.ensemble import GradientBoostingRegressor
from statsmodels.api import GLM
from statsmodels.api import families


In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import linregress
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import numpy as np
import pandas as pd
import math
import shap
import warnings
import scipy
from scipy.stats import pearsonr

def mean_directional_accuracy_test(y_true, y_pred):

    try:
        return np.mean(np.sign(y_true[1:] - y_true[:-1]) == np.sign(y_pred[1:] - y_pred[:-1]))
    except:
        differences = np.array(y_pred) - np.array(y_true) 
        signs = np.sign(differences)
        mde = np.mean(signs)
        return mde
        


from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import linregress
import numpy as np
import pandas as pd
import math
import shap
import warnings
from skopt import BayesSearchCV
from skopt.space import Real, Integer

def Regression_GBR_nested(X, y, outer_splits=10, inner_splits=5, shaps_comp=False):
    warnings.filterwarnings("ignore")

    scaler = MinMaxScaler((0.05, 0.95))

    outer_cv = KFold(n_splits=outer_splits, shuffle=True, random_state=42)
    param_grid = {
        'n_estimators': Integer(50, 500),          # Número de árboles
        'learning_rate': Real(0.01, 0.5, prior='log-uniform'),  # Tasa de aprendizaje
        'max_depth': Integer(1, 5),               # Profundidad máxima de los árboles
        'min_samples_split': Integer(2, 10),       # Mínimo de muestras para dividir un nodo
        'min_samples_leaf': Integer(1, 10),        # Mínimo de muestras en una hoja
    }

    y_labels = []
    y_predicts = []
    r_squared_l = []
    mse_l = []
    rmse_l = []

    results_labels_df = pd.DataFrame(columns=['y_labels', 'y_pred', 'GAP', 'GAP_corrected', 'ID'])

    coef_array = np.zeros([X.shape[1] + 1, outer_splits])
    lista_vars = list(X.columns)

    for fold, (train_idx, test_idx) in enumerate(outer_cv.split(X)):

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


        scaling_data = scaler.fit_transform(X_train)
        X_train = pd.DataFrame(scaling_data, columns= X_train.columns, index = X_train.index)

        scaling_data = scaler.transform(X_test)
        X_test = pd.DataFrame(scaling_data, columns= X_test.columns, index = X_test.index)
        
        model = GradientBoostingRegressor(random_state=42)
        inner_cv = KFold(n_splits=inner_splits, shuffle=True, random_state=42)
        
        bayes_search = BayesSearchCV(
            estimator=model,
            search_spaces=param_grid,
            n_iter=3,              
            scoring='r2',  
            cv=inner_cv,                   
            n_jobs=-1,             
            random_state=42
        )
        bayes_search.fit(X_train, y_train)

        best_model = bayes_search.best_estimator_

        y_pred = best_model.predict(X_test)
        gap_test = y_pred - y_test
        gap_train = best_model.predict(X_train) - y_train

        slope, intercept, _, _, _ = linregress(y_train, gap_train)
        corrected_gap = gap_test - (slope * y_test + intercept)

        result = np.column_stack((y_test, y_pred, gap_test, corrected_gap))
        temp_df = pd.DataFrame(result, columns=['y_labels', 'y_pred', 'GAP', 'GAP_corrected'])
        temp_df['ID'] = X_test.index
        results_labels_df = pd.concat([results_labels_df, temp_df], ignore_index=True)

        y_labels.extend(y_test)
        y_predicts.extend(y_pred)

        r_squared_l.append(r2_score(y_test, y_pred))
        mse_l.append(mean_squared_error(y_test, y_pred))
        rmse_l.append(math.sqrt(mse_l[-1]))

        coef_array[0, fold] = np.nan
        coef_array[1:, fold] = best_model.feature_importances_

    # Métricas finales
    y_labels = np.array(y_labels)
    y_predicts = np.array(y_predicts)

    r_squared = r2_score(y_labels, y_predicts)
    r_squared_ = np.mean(r_squared_l)
    n = len(y_labels)
    p = X.shape[1]
    k = p - 1

    r_squared_adj = 1 - (1 - r_squared) * (n - 1) / (n - k - 1)
    mse = np.mean(mse_l)
    rmse = np.mean(rmse_l)
    mae = mean_absolute_error(y_labels, y_predicts)
    F = (r_squared / p) / ((1 - r_squared) / (n - p - 1))
    p_value = np.round(scipy.stats.f.sf(F, n, (n - p - 1)), 15)
    F2 = r_squared / (1 - r_squared)

    # Coeficientes medios
    coef_df = pd.DataFrame(index=['_intercept'] + lista_vars,
                           columns=['Estimate mean', 'Estimate std', 't value', 'p value'])

    coef_df['Estimate mean'] = coef_array.mean(axis=1)
    coef_df['Estimate std'] = coef_array.std(axis=1)
    #coef_df['t value'] = coef_tval(coef_array.mean(axis=1), X, y_labels, y_predicts)
    #coef_df['p value'] = coef_pval(coef_array.mean(axis=1), X, y_labels, y_predicts)

    coef_df.loc['_intercept', 'R2'] = r_squared
    coef_df.loc['_intercept', 'R2 adj'] = r_squared_adj
    coef_df.loc['_intercept', 'R2 [+-]'] = np.std(r_squared_l)
    coef_df.loc['_intercept', 'F2'] = F2
    coef_df.loc['_intercept', 'mse'] = mse
    coef_df.loc['_intercept', 'mse [+-]'] = np.std(mse_l)
    coef_df.loc['_intercept', 'rmse'] = rmse
    coef_df.loc['_intercept', 'rmse [+-]'] = np.std(rmse_l)
    coef_df.loc['_intercept', 'outcome var'] = np.var(y)
    coef_df.loc['_intercept', 'F'] = F
    coef_df.loc['_intercept', 'F-p_value'] = p_value
    coef_df.loc['_intercept', 'MDE'] = mean_directional_accuracy(y_labels, y_predicts)
    coef_df.loc['_intercept', 'MAE'] = mae

    results_labels_df['y_pred_corrected'] = results_labels_df['y_labels'] + results_labels_df['GAP_corrected']
    results_labels_df = results_labels_df[['ID','y_labels', 'y_pred', 'GAP', 'GAP_corrected', 'y_pred_corrected']]

    if shaps_comp:
        explainer = shap.Explainer(best_model, X)
        return [coef_df, r_squared_, results_labels_df, explainer]
    else:
        return [coef_df, r_squared_, results_labels_df]


## Load data

In [ ]:
data = pd.read_csv('data/BBAG-exta-vars.csv', index_col= 0)
data

In [ ]:
vars_list = ['Sex_1F_2M',
              'Education',
              'Barthel',
              'Diabetes_1Y_0N',
              'Hypertension_1Y_0N',
              'Heart_Disease_1Y_0N',
              'Physical_activity_1Y_0N',
              'Cognition',
              'Weight_problems_v02_1Y_0N',
              'Well_being_domain',
              'Sleep_problems_1Y_0N',
              'Audition_problems', 
              'Vision_problems', 
              'Alcohol_consumption_1Y_0N']

In [ ]:
len(vars_list)

## BBAG model

In [ ]:
X1 = data[vars_list]
y1 = data['Age']


[coef_df_, r_squared_, results_labels_df_] = Regression_GBR_nested(X1, y1, outer_splits=10, inner_splits=5, shaps_comp=False)

In [ ]:
display(coef_df_)

In [ ]:
results_GB = data.copy()
results_GB.index.name = "ID"

results_GB_ = results_labels_df_.set_index('ID')
final_dataframe = pd.merge(results_GB_, results_GB, on='ID', how='inner')

In [ ]:
final_dataframe.columns

In [ ]:
#final_dataframe.to_csv('name.csv')